In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 기후 데이터 확인

In [2]:
# 날씨 정보 가져오기

weather_data = pd.read_csv("data-files/weather.csv", encoding="cp949")

In [3]:
weather_data

,지점,지점명,일시,평균기온(°C),평균풍속(m/s),월강수량합(mm)
0,12,안면도(감),2019-01,1.3,2.6,0.5
1,12,안면도(감),2019-02,1.7,2.6,15.0
2,12,안면도(감),2019-03,6.1,2.6,28.0
3,12,안면도(감),2019-04,10.4,2.1,49.5
4,12,안면도(감),2019-05,15.8,2.7,19.0
...,...,...,...,...,...,...
36805,996,화동,2024-06,21.4,2.5,122.0
36806,996,화동,2024-07,24.3,2.5,584.0
36807,996,화동,2024-08,25.8,2.3,34.5
36808,996,화동,2024-09,22.6,2.4,208.5


# 관측지 정보 데이터 확인

In [4]:
# 관측지 정보 가져오기

station_meta_data = pd.read_csv("data-files/관측지점정보.csv", encoding='cp949')

In [5]:
station_meta_data

,지점,시작일,종료일,지점명,지점주소,관리관서,위도,경도,노장해발고도(m),기압계(관측장비지상높이(m)),기온계(관측장비지상높이(m)),풍속계(관측장비지상높이(m)),강우계(관측장비지상높이(m))
0,12,2007-11-30,NaN,안면도(감),NaN,NaN,36.5333,126.3167,60.00,NaN,NaN,NaN,NaN
1,96,2020-10-23,NaN,독도,경상북도 울릉군울릉읍 독도이사부길63,울릉도기상대(115),37.2395,131.8698,99.08,NaN,NaN,NaN,NaN
2,96,2009-10-27,2020-10-23,독도,경상북도 울릉군울릉읍 독도이사부길63,울릉도기상대(115),37.2395,131.8698,96.15,NaN,NaN,NaN,NaN
3,116,2023-10-21,NaN,관악(레),경기도 과천시중앙로 관악산길(관악산기상레이더관측소),NaN,37.4453,126.9640,624.82,NaN,NaN,NaN,NaN
4,116,2010-08-06,2023-10-21,관악(레),경기도 과천시중앙로 관악산길(관악산기상레이더관측소),NaN,37.4453,126.9640,624.82,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2413,990,2022-10-13,NaN,낙천,NaN,제주지방기상청(184),33.3182,126.2305,77.67,NaN,NaN,NaN,NaN
2414,990,2021-11-17,2022-10-13,낙천,NaN,제주지방기상청(184),33.3182,126.2305,76.00,NaN,NaN,NaN,NaN
2415,991,2021-11-10,NaN,옥포,NaN,대구(구 143),35.8028,128.4456,27.50,NaN,NaN,NaN,NaN
2416,992,2021-11-10,NaN,하빈,대구광역시 달성군하빈면 현내리 325-3 (국립식량과학원 대구시험지),대구(구 143),35.9062,128.4464,35.50,NaN,NaN,NaN,NaN


# 관측지 정보 데이터로 기후 데이터의 광역시도 처리

In [ ]:
# 관측지 정보의 지점주소 컬럼의 첫번째 단어가져와서 광역시도로 저장

station_meta_data['광역시도'] = station_meta_data['지점주소'].str.split().str[0]

# weather_data의 모든 데이터를 포함하며, station_meta_data에서 일치하는 데이터만 추가
merged_weather_data = weather_data.merge(
    station_meta_data[['지점', '광역시도']],
    on='지점',
    how='left'
)

# 연도, 월로 저장
merged_weather_data['연도'] = pd.to_datetime(merged_weather_data['일시']).dt.year
merged_weather_data['월'] = pd.to_datetime(merged_weather_data['일시']).dt.month


# 그룹화
grouped_weather_data = merged_weather_data.groupby(['광역시도', '연도', '월']).agg({
    '평균기온(°C)': 'mean',
    '평균풍속(m/s)': 'mean',
    '월강수량합(mm)': 'mean'
}).reset_index()

# 컬럼이름 변경
grouped_weather_data.rename(columns={
    '평균기온(°C)': '평균기온',
    '평균풍속(m/s)': '평균풍속',
    '월강수량합(mm)': '평균강수량'
}, inplace=True)

# 기후 데이터의 추가한 광역시도 컬럼 확인

In [ ]:
# 광역시도 컬럼 확인

grouped_weather_data['광역시도'].unique()

array(['강원특별자치도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시',
       '부산광역시', '서귀포시', '서울특별시', '세종특별자치시', '울산광역시', '인천광역시', '전라남도',
       '전북특별자치도', '제주특별자치도', '충청남도', '충청북도'], dtype=object)

# 이름이 다른 광역시도 처리

In [ ]:
# 컬럼명 변경

grouped_weather_data['광역시도'] = grouped_weather_data['광역시도'].replace({
    '전북특별자치도': '전라북도',
    '강원특별자치도': '강원도',
    '제주특별자치도': '제주도',
    '서귀포시': '제주도',
    '세종특별자치시' : '세종시'
})

In [ ]:
# 소수점 2자리

grouped_weather_data = grouped_weather_data.round(2)

In [59]:
grouped_weather_data

,광역시도,연도,월,평균기온,평균풍속,평균강수량
0,강원도,2019,1,-3.48,2.06,2.07
1,강원도,2019,2,-1.12,1.82,22.44
2,강원도,2019,3,4.30,2.18,41.57
3,강원도,2019,4,9.15,2.13,55.95
4,강원도,2019,5,16.84,2.24,18.88
...,...,...,...,...,...,...
1193,충청북도,2024,6,22.34,1.27,118.63
1194,충청북도,2024,7,25.83,1.41,449.82
1195,충청북도,2024,8,27.35,1.10,67.85
1196,충청북도,2024,9,23.90,1.08,233.70


In [ ]:
# grouped_weather_data.to_csv("data-files/group_weather.csv", index=False)

In [67]:
grouped_weather_data = pd.read_csv("data-files/group_weather.csv")

# 연도와 월을 합쳐서 DATE 컬럼으로

In [ ]:
# 연도, 월을 DATE컬럼으로 합치기

grouped_weather_data['DATE'] = pd.to_datetime(grouped_weather_data['연도'].astype(str) + '-' + grouped_weather_data['월'].astype(str), format='%Y-%m')

In [ ]:


grouped_weather_data = grouped_weather_data.drop(columns=['연도', '월'])

In [73]:
grouped_weather_data

,광역시도,평균기온,평균풍속,평균강수량,DATE
0,강원도,-3.48,2.06,2.07,2019-01-01
1,강원도,-1.12,1.82,22.44,2019-02-01
2,강원도,4.30,2.18,41.57,2019-03-01
3,강원도,9.15,2.13,55.95,2019-04-01
4,강원도,16.84,2.24,18.88,2019-05-01
...,...,...,...,...,...
1193,충청북도,22.34,1.27,118.63,2024-06-01
1194,충청북도,25.83,1.41,449.82,2024-07-01
1195,충청북도,27.35,1.10,67.85,2024-08-01
1196,충청북도,23.90,1.08,233.70,2024-09-01


In [74]:
grouped_weather_data['DATE'] = pd.to_datetime(grouped_weather_data['DATE']).dt.strftime('%Y-%m')

In [ ]:
# grouped_weather_data.to_csv("data-files/weather_final.csv", index=False)